# A specutils example for GBspec

This version uses the SpectrumList container. Could also consider a list of Spectrum1D or a SpectrumCollection.

This should reproduce Example 1 from the GBTIDL manual. The datafile you need is [here](http://safe.nrao.edu/wiki/pub/GB/Data/GBTIDLExampleAndSampleData/ngc5291.fits).

## SDFITS

We begin by disecting the typical SDFITS file, starting with raw plotting of a spectrum.



In [ ]:
# %matplotlib inline
%matplotlib

from astropy.io import fits
from astropy import units as u
import numpy as np
from matplotlib import pyplot as plt
from astropy.visualization import quantity_support
from specutils import Spectrum1D, SpectrumList

First a useful statistics routine, we can also use it for regression


In [ ]:
def my_stats(label,data):
    """
    display mean,rms,min,max
    """
    mean = data.mean()
    rms  = data.std()
    dmin = data.min()
    dmax = data.max()
    print("%s  %s %s %s %s" %  (label,repr(mean),repr(rms),repr(dmin),repr(dmax)))

define the SDFITS file name and which row (0 being the first) we want to plot the spectrum of

In [ ]:
fname = 'ngc5291.fits'
row   = 0

Open the FITS file and point to the 2nd HDU, where the BINTABLE is located. No error checking

In [ ]:
hdu = fits.open(fname)
header2 = hdu[1].header
data2   = hdu[1].data
nrow = len(data2)
print("Found %d rows" % nrow)

Grab spectrum by row number. Get some stats (mean, rms, min, max)

In [ ]:
flux  = data2[row]['DATA']  
nchan = len(flux)
chans = np.arange(nchan)
print("Found %d channels" % nchan)
#
my_stats('STATS for row %d:' % row,flux)

A super simple plot, channel number vs. flux

In [ ]:
plt.plot(chans,flux)
plt.xlabel("Channel")
plt.ylabel("Flux");

This is a raw spectrum, mostly showing sky and bandpass.Hidden is a tiny signal, not even visible in the plot. For this we need to collect an "On" and "Off" spectrum and normalize this difference. Row 0 is the "on" and row 1 the "off", this pattern repeats.

### A few words on row organization

All the magic of calibration is in the organization and labeling of the rows. For the **position switching** of this data we will see 352 rows organized as follows:

    DATA(cal,int,sampler,procseqn,scan)
          2   11    2        2      4
          
with the following notes:

* DATA is labeled in column-major ("fortran") order, i.e. the first listed dimension runs fastest
* 2 * 11 * 2 * 2 * 4 = 352
* each procseqn also increments the scan number, so there are really 8 scan values but only 4 in that dimension
* each sampler is taken at the same time, so this means after 22 rows (cal * int) , time repeats in the next 
  sampler (the XX and YY pol in this case)
* in this data samplers point to different polarizations, in argus samplers are the beams/feeds for a single XX polarization
  

### How about Pandas DataFrame's ?

A pandas dataframe could contain the whole BINTABLE, and with an attached 'engine' reduce the rows in the waterfall plot to lower dimension (eventually 1 if it's a single pointing), which is the desired result. 
Compare this with operators such as **np.sum(axis=1)/nrow** if you just want to get an average spectrum.

In [ ]:
from astropy.table import Table
import pandas

t= Table.read(fname,format='fits') 
if 'TDIM7' in t.colnames:
    print("Expect trouble, there is TDIM7")
#t.meta
df = t.to_pandas()
 
# problem is that TDIM7 = '(32768,1,1,1,)' so we should comment out TDIM7 -> XTDIM7 or remove column (e.g. in fv)
# hmm, still doesn't work

### Adding astropy units to the X and Y axis

specutils likes to work with things that have units. astropy to the rescue

In [ ]:
crval1 = data2[row]['CRVAL1']
cdelt1 = data2[row]['CDELT1']
crpix1 = data2[row]['CRPIX1']
freq0  = data2[row]['RESTFREQ']
freq = (crval1 + (np.arange(1,nchan+1) - crpix1) * cdelt1)/1e9
#
flux = flux * u.Unit("K")      # where is this in the header, we're assuming
freq = freq * u.Unit("GHz")    # or check CUNIT1 - usually not present in SDFITS files though

In [ ]:
# even though we've attached astropy units, they still work in matplotlib plotting
plt.plot(freq,flux)
plt.xlabel("Freq [%s]" % freq.unit)
plt.ylabel("Flux [%s]" % flux.unit);

### Creating a simple Spectrum1D object

In [ ]:
spec = Spectrum1D(spectral_axis=freq, flux=flux)

In [ ]:
f, ax = plt.subplots()  
ax.step(spec.spectral_axis, spec.flux);
# darn, the spectral axis still has 1e9 units, and it doesn't identify the units

## Designing our own reader in specutils

The specutils manual explains how to make your own reader. At the moment of writing there is no "sdfits" reader, so we're making a simple example with just a few meta-data. But this will simplify working with specutils.

In [ ]:
from specutils.io import get_loaders_by_extension

loaders = get_loaders_by_extension('fits')
print(loaders)

In [ ]:
from astropy.nddata import StdDevUncertainty
from astropy.table import Table
from astropy.units import Unit
from astropy.wcs import WCS

from specutils.io.registers import data_loader
from specutils import Spectrum1D


We need a list of FITS keywords and FIELD names from the BINTABLE that are going to be the meta-data
associated with each spectrum.


In [ ]:
# just a few important ones for now, there are about 70 in the full SDFITS
sdfits_headers = ['SCAN', 'PROCSEQN', 'CAL', 'OBJECT','SAMPLER', 'TSYS']



We are registering a special fits reader for the spectra, and use a SpectrumList.


In [ ]:
def identify_sdfits(origin, *args, **kwargs):
    print("IDENTIFY_SDFITS")
    try:
        with fits.open(args[0]) as hdulist:
            extname = hdulist[1].header['EXTNAME']
            if extname == 'SINGLE DISH':
                print("Hurray, we have SDFITS")
                return True
            else:
                print("Warning, skipping extname %s" % extname)
                return False
    except Exception:
        return Falseflux + sl2[i].flux

    

In [ ]:
@data_loader("sdfits", identifier=identify_sdfits, dtype=SpectrumList, extensions=['fits'])
def sdfits_loader(file_name, spectral_axis_unit=None, **kwargs):

    spectra = []
    with fits.open(file_name, **kwargs) as hdulist:
        header1= hdulist[0].header        
        header2= hdulist[1].header
        data   = hdulist[1].data
        nrow   = len(data)
        nchan  = 0
        for i in range(nrow):
            sp = data[i]['DATA']
            if nchan==0:
                nchan = len(sp)     # every spectrum in SDFITS has the same length
            crval1  = data[i]['CRVAL1']
            cdelt1  = data[i]['CDELT1']
            crpix1  = data[i]['CRPIX1']
            ctype1  = data[i]['CTYPE1']     # 'FREQ-OBS' to 'FREQ'; assuming SPECSYS='TOPOCENT'
            restfrq = data[i]['RESTFREQ']
            cunit1  = 'Hz'
            crval2  = data[i]['CRVAL2']
            crval3  = data[i]['CRVAL3']
            ctype2  = data[i]['CTYPE2']
            ctype3  = data[i]['CTYPE3']
            if ctype1 == 'FREQ-OBS': ctype1  = 'FREQ'
            wcs = WCS(header={'CDELT1': cdelt1, 'CRVAL1': crval1, 'CUNIT1': cunit1,
                              'CTYPE1': ctype1, 'CRPIX1': crpix1, 'RESTFRQ': restfrq,
                              'CTYPE2': ctype2, 'CRVAL2': crval2,
                              'CTYPE3': ctype3, 'CRVAL3': crval3})
                              

            meta = {}
            if False:
                # adding the actual FITS headers is for debugging, but not in production mode
                meta['header1'] = header1
                meta['header2'] = header2
            if True:
                for key in sdfits_headers:
                    if key in header1:
                        meta[key] = header1[key]
                    elif key in header2:
                        meta[key] = header2[key]
                    else:
                        meta[key] = data[i][key]    # why doesn't       key in data[i]    work?
                # add our row counter
                meta['_row'] = i
                        
        
            sp = sp * Unit('K')
            spec = Spectrum1D(flux=sp, wcs=wcs, meta=meta)
            spectra.append(spec)
            
    return  SpectrumList(spectra)

Now we are ready for some action! But lets define a small spectrum plotter with some nicer units



In [ ]:
def my_plot(sp, kms=False):
    """ plot a Spectrum1D 
    """
    if kms:
        x = sp.velocity   # doesn't work
    else:
        x = sp.spectral_axis / (1 * Unit("Hz"))/1e6
    if False:
        plt.plot(x, sp.flux)
        plt.xlabel('Frequency [MHz]')
        plt.ylabel('Flux [%s]' % sp.flux.unit)
    else:
        f, ax = plt.subplots()  # doctest: +IGNORE_OUTPUT
        ax.step(sp.spectral_axis, sp.flux) 

In [ ]:
sl1 = SpectrumList.read(fname, format="sdfits")
nsp = len(sl1)
print("Found %d spectra" % nsp)
print("Here are the first two spectra, the On and Off")
for i in range(2):
    my_stats("test",sl1[i].flux.value)

In [ ]:
sp  = sl1[0]
x=sp.spectral_axis/(1 * Unit("Hz"))/1e6
y=sp.flux
type(y)
print(y.unit)


In [ ]:
a=sl1[0]-sl1[1]
b=sl1[2]-sl1[3]
print(a)
print(b)
try:
    c = a + b
except:
    print("*** Cannot combine ***")
a1 = a / sl1[0]
print(a1)
# cannot do a+b, spectral axis is TOPO
# does this imply that if our ephemeris isn't good enough, they will not align in doppler space
# and thus refuse to be combined?

In [ ]:
# lets look at the meta data where we know there are changes
# there are 8 scans of 11 integrations each, two polarizations and the on/off cal cycle
for i in [0,1,22,44,88]:
    print(i,sl1[i].meta)


In [ ]:
spectra = []
for i in range(0,nsp,2):
    spon  = sl1[i]         
    spoff = sl1[i+1]
    tsys  = sl1[i].meta['TSYS']
    sp = (spon-spoff)/spoff.flux.value        # trick: otherwise it's loosing the units here
    sp.meta = spon.meta
    #sp.flux *= tsys           # this is silly, can't modify the flux.....
    spectra.append(sp)
sl2 =  SpectrumList(spectra)   

Take the first calibrated spectrum, and see how it compares ith GBTIDL

In [ ]:
# GBTIDL:  getps,51,intnum=0  &  stats,/full
# 0.27865 0.16342 -1.5897 1.8587
my_stats("first R:",sl2[0].flux.value)
my_plot(sp)

In [ ]:
print(sp)
a=sp.multiply(2*u.Unit('mJy'))   
print(sp)
print(a)
a = a * (2*u.Unit('mJy'))
print(a)


In [ ]:
# just for fun, plot the last On-Off
spd = spon - spoff
plt.plot(sl2[0].spectral_axis, spd.flux);

In [ ]:
for i in [0,1,11,22]:
    print(i,sl2[i].meta)

In [ ]:
for i in range(len(sl2)):
    plt.plot(sl2[i].spectral_axis, sl2[i].flux)

In [ ]:
# this is wrong, but i don't know why yet, noise seems to high too
# something more must be going on in the getps() procedure
# I'm averaging 8 scans here, but getps() example suggests only the odd ones?
sl3 = []
flux = sl2[0].flux * 0.0
for i in range(len(sl2)):
    flux = flux + sl2[i].flux
flux = flux / len(sl2)   
plt.plot(sl2[0].spectral_axis,flux);
        

In [ ]:
sp = sl2[0]
sp = sp.multiply(0*u.Unit('Jy'))     # the unit doesn't appear to be import, but it needs a unit
print(sp)

In [ ]:
for i in range(len(sl2)):
    sp = sp.add(sl2[i])

In [ ]:
from specutils.manipulation import (box_smooth, gaussian_smooth, trapezoid_smooth)
spec1_bsmooth = box_smooth(spec1, width=3)
spec1_gsmooth = gaussian_smooth(spec1, stddev=3)
spec1_tsmooth = trapezoid_smooth(spec1, width=3)

